In [4]:
!pip3 install elasticsearch



    100% |████████████████████████████████| 194kB 1.7MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/5e/c4/6c4fe722df5343c33226f0b4e0bb042e4dc13483228b4718baf286f86d87/certifi-2020.6.20-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9f/f0/a391d1463ebb1b233795cabfc0ef38d3db4442339de68f847026199e69d7/urllib3-1.25.10-py2.py3-none-any.whl


In [5]:
import requests
data = requests.get('http://localhost:9200')
print(data.content)


b'{\n  "name" : "niharika-ambarisvr",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "rJnqJfQsSa-4Acn0uy4okw",\n  "version" : {\n    "number" : "7.8.0",\n    "build_flavor" : "default",\n    "build_type" : "deb",\n    "build_hash" : "757314695644ea9a1dc2fecd26d1a43856725e65",\n    "build_date" : "2020-06-14T19:35:50.234439Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.5.1",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [6]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])


In [7]:
import pandas as pd
dataf = pd.read_csv("/home/niharika/Documents/covid/metadata.csv")


/home/niharika/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
dataf.columns


Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')

In [9]:
result = es.search(index = "metafile",body = {
            "query": { 
                "bool": { 
                    "must": [
                        { "match": {"journal" : "J Exot Pet Med" }}
                  ],
                }
              }
            })


In [10]:
result

{'took': 4760,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 20.780575,
  'hits': [{'_index': 'metafile',
    '_type': '_doc',
    '_id': 'Daqv-HMB270E1nsYeiBB',
    '_score': 20.780575,
    '_source': {'sha': None,
     '@version': '1',
     'source_x': 'PMC',
     'title': 'Avian & Exotic News',
     'path': '/home/niharika/Documents/covid/metadata.csv',
     'pmc_json_files': None,
     'mag_id': None,
     'pmcid': 'PMC7129310',
     'publish_time': '2016-05-21',
     'authors': 'Orcutt, Connie',
     'doi': '10.1053/j.jepm.2016.03.016',
     'cord_uid': 'wz3porkn',
     'pdf_json_files': None,
     'abstract': None,
     'host': 'niharika-ambarisvr',
     'pubmed_id': None,
     'license': 'no-cc',
     'message': 'wz3porkn,,PMC,Avian & Exotic News,10.1053/j.jepm.2016.03.016,PMC7129310,,no-cc,,2016-05-21,"Orcutt, Connie",J Exot Pet Med,,,,,,https://www.ncbi.nlm.nih.g

In [11]:
source = es.search(index = "metafile",body = {
            "query": { 
                "bool": { 
                    "must": [
                        { "match": {'source_x': 'PMC'}}
                  ],
                }
              }
            })


In [12]:
source

{'took': 1385,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.1287017,
  'hits': [{'_index': 'metafile',
    '_type': '_doc',
    '_id': '0tEu-HMBhlvJXOOCCcGw',
    '_score': 1.1287017,
    '_source': {'pubmed_id': '20573630',
     'pmc_json_files': 'document_parses/pmc_json/PMC3033019.xml.json',
     'abstract': 'There is still limited understanding of key determinants of spatial spread of influenza. The 1918 pandemic provides an opportunity to elucidate spatial determinants of spread on a large scale. To better characterize the spread of the 1918 major wave, we fitted a range of city-to-city transmission models to mortality data collected for 246 population centres in England and Wales and 47 cities in the US. Using a gravity model for city-to-city contacts, we explored the effect of population size and distance on the spread of disease and tested assumptions regarding

In [13]:
pub_date = es.search(index="metafile", body={"_source": ["title"], 
                                     "query": {"range": 
                                               {"publish_time": 
                                                {"lt":"2020-01-01"}}}})


In [14]:
pub_date

{'took': 842,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.0,
  'hits': [{'_index': 'metafile',
    '_type': '_doc',
    '_id': '0tEu-HMBhlvJXOOCCcGw',
    '_score': 1.0,
    '_source': {'title': 'Spatial dynamics of the 1918 influenza pandemic in England, Wales and the United States'}},
   {'_index': 'metafile',
    '_type': '_doc',
    '_id': '09Eu-HMBhlvJXOOCCcGw',
    '_score': 1.0,
    '_source': {'title': 'Charge-Surrounded Pockets and Electrostatic Interactions with Small Ions Modulate the Activity of Retroviral Fusion Proteins'}},
   {'_index': 'metafile',
    '_type': '_doc',
    '_id': '1NEu-HMBhlvJXOOCCcGw',
    '_score': 1.0,
    '_source': {'title': 'Tracheostomy and mechanical ventilation weaning in children affected by respiratory virus according to a weaning protocol in a pediatric intensive care unit in Argentina: an observational restrospective trial'

In [15]:
es.count(index='metafile')


{'count': 334072,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [16]:
es.info()

{'name': 'niharika-ambarisvr',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'rJnqJfQsSa-4Acn0uy4okw',
 'version': {'number': '7.8.0',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '757314695644ea9a1dc2fecd26d1a43856725e65',
  'build_date': '2020-06-14T19:35:50.234439Z',
  'build_snapshot': False,
  'lucene_version': '8.5.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}